In [2]:
import pandas as pd

# Load the data
df_final_web_data_pt_1 = pd.read_csv('../1_data/df_final_web_data_pt_1.txt')
df_final_web_data_pt_2 = pd.read_csv('../1_data/df_final_web_data_pt_2.txt')
df_final_experiment_clients = pd.read_csv('../1_data/df_final_experiment_clients.txt')

# Merge web data
df_web_cleaned = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)


# Step 1: Remove exact duplicates based on 'visit_id', 'process_step', and 'date_time'
duplicates_before = df_web_cleaned.shape[0]
df_web_cleaned = df_web_cleaned.drop_duplicates(subset=['visit_id', 'process_step', 'date_time'], keep='first')  # Remove exact duplicates
duplicates_after = df_web_cleaned.shape[0]
print(f"Dropped {duplicates_before - duplicates_after} duplicate rows from df_web.")

# Step 2: Handle missing values in essential columns ('client_id', 'visit_id', and 'process_step')
missing_values_before = df_web_cleaned.shape[0]
df_web_cleaned = df_web_cleaned.dropna(subset=['client_id', 'visit_id', 'process_step'])
missing_values_after = df_web_cleaned.shape[0]
print(f"Dropped {missing_values_before - missing_values_after} rows with missing values from df_web.")

# Step 3: Replace 'confirm' and 'start' with 'step_4' and 'step_0' for process_step
df_web_cleaned['process_step'] = df_web_cleaned['process_step'].replace('confirm', 'step_4')
df_web_cleaned['process_step'] = df_web_cleaned['process_step'].replace('start', 'step_0')

# Step 4: Convert 'date_time' to datetime format
df_web_cleaned['date_time'] = pd.to_datetime(df_web_cleaned['date_time'])

# Step 5: Sort data by 'visit_id', 'process_step', and 'date_time'
df_time = df_web_cleaned.sort_values(by=['visit_id', 'process_step', 'date_time'])

# Print the number of rows after cleaning
print(f"Final rows in df_time: {df_time.shape[0]}")
print("Cleaned df_time:")
print(df_time.head())





Dropped 10764 duplicate rows from df_web.
Dropped 0 rows with missing values from df_web.
Final rows in df_time: 744641
Cleaned df_time:
        client_id             visitor_id                      visit_id  \
108614    3561384   451664975_1722933822  100012776_37918976071_457913   
108613    3561384   451664975_1722933822  100012776_37918976071_457913   
447951    9056452  306992881_89423906595     1000165_4190026492_760066   
447950    9056452  306992881_89423906595     1000165_4190026492_760066   
447949    9056452  306992881_89423906595     1000165_4190026492_760066   

       process_step           date_time  
108614       step_4 2017-04-26 13:22:17  
108613       step_4 2017-04-26 13:23:09  
447951       step_0 2017-06-04 01:07:29  
447950       step_1 2017-06-04 01:07:32  
447949       step_2 2017-06-04 01:07:56  


In [3]:
# Merge df_web_cleaned with df_final_experiment_clients on 'client_id' to include 'Variation'
df_time = df_web_cleaned.merge(df_final_experiment_clients[['client_id', 'Variation']], on='client_id', how='left')

# Drop rows where 'Variation' is neither 'Test' nor 'Control'
df_time = df_time[df_time['Variation'].isin(['Test', 'Control'])]

# Check the result
print(f"Number of rows in df_time after filtering: {df_time.shape[0]}")
print(df_time.head())


Number of rows in df_time after filtering: 317235
   client_id            visitor_id                      visit_id process_step  \
0    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
1    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
2    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   
3    9988021  580560515_7732621733  781255054_21935453173_531117       step_2   
4    9988021  580560515_7732621733  781255054_21935453173_531117       step_3   

            date_time Variation  
0 2017-04-17 15:27:07      Test  
1 2017-04-17 15:26:51      Test  
2 2017-04-17 15:19:22      Test  
3 2017-04-17 15:19:13      Test  
4 2017-04-17 15:18:04      Test  


### KPI 1: Completion Rate (reaching step_4)

##### Selecting users that went through at least one step before confirmation ##### 

In [4]:
# Step 1: Identify visits that only show step_4
step_counts = df_time.groupby('visit_id')['process_step'].nunique().reset_index(name='unique_steps')
step_4_only_visits = df_time[df_time['process_step'] == 'step_4']['visit_id'].unique()
step_4_only = step_counts[(step_counts['unique_steps'] == 1) & (step_counts['visit_id'].isin(step_4_only_visits))]['visit_id']

# Step 2: Exclude them from df_time
df_time_filtered = df_time[~df_time['visit_id'].isin(step_4_only)]

print(f"Excluded {len(step_4_only)} visits that only had step_4.")
print(f"Remaining visits: {df_time_filtered['visit_id'].nunique()}")


Excluded 4120 visits that only had step_4.
Remaining visits: 65085


In [9]:
# Step 3: Identify which visits reached step_4 (confirmation)
completed_visits = df_time_filtered[df_time_filtered['process_step'] == 'step_4']['visit_id'].unique()

# Step 4: Get all remaining visits by Variation
visit_variations = df_time_filtered[['visit_id', 'Variation']].drop_duplicates()

# Step 5: Mark visits as completed or not
visit_variations['completed'] = visit_variations['visit_id'].isin(completed_visits)

# Step 6: Completion rate by Variation
completion_summary = visit_variations.groupby('Variation')['completed'].agg(['sum', 'count'])
completion_summary['completion_rate'] = (completion_summary['sum'] / completion_summary['count']) * 100
completion_summary = completion_summary.rename(columns={'sum': 'completions', 'count': 'total_visits'})
completion_summary = completion_summary.reset_index()

completion_summary


,Variation,completions,total_visits,completion_rate
0,Control,15296,31428,48.669976
1,Test,18376,33776,54.405495


In [28]:
completion_summary.to_csv(r'C:\Users\manoe.MC_ASUS\Documents\IRON HACK BOOTCAMP\DAFT_Feb 2025\projects\Project 5\Project-5---Vanguard\1_data\tableau\completion_rate_summary.csv', index=False)



### HYPOTHESIS TEST: COMPLETION RATE KPI

In [16]:
from statsmodels.stats.proportion import proportions_ztest

# Successes 
successes = [18376, 15296]
totals = [33776, 31428]

# z-test
z_stat, p_val = proportions_ztest(count=successes, nobs=totals, alternative='larger')


formatted_p = f"{p_val:.2e}"  # or use .3e for 3 decimals
formatted_z = f"{z_stat:.2f}"

print(f"Z-statistic: {formatted_z}")
print(f"P-value: {formatted_p}")



Z-statistic: 14.64
P-value: 7.35e-49


##### CONCLUSION: 

Z-statistic: 14.6441

P-value: virtually 0 (7.35e-49)

Completion rate difference:

Control: 48.67%

Test: 54.41%

Difference: ~5.74 percentage points 

The new UI led to a statistically significant increase in completion rate of ~5.7 percentage points, suggesting the Test version performs better.m

### KPI 2: Average time per step

In [17]:
df_time = df_time.sort_values(by=['visit_id', 'date_time'])

df_time['diff_time'] = df_time.groupby('visit_id')['date_time'].diff()


In [18]:
# clean the time column
df_time = df_time[df_time['diff_time'].notna()]
df_time = df_time[df_time['diff_time'] >= pd.Timedelta(0)]

# Filter to only test and control clients
df_ab_time = df_time[df_time['Variation'].isin(['Test', 'Control'])]

# Compute mean time per step for each Variation
avg_time_per_step = df_ab_time.groupby(['Variation', 'process_step'])['diff_time'].mean().reset_index()

# Optional: Format time nicely
avg_time_per_step['formatted_time'] = avg_time_per_step['diff_time'].apply(lambda x: str(x).split('.')[0])

# Display result
print(avg_time_per_step[['Variation', 'process_step', 'formatted_time']])


  Variation process_step   formatted_time
0   Control       step_0  0 days 00:03:19
1   Control       step_1  0 days 00:00:56
2   Control       step_2  0 days 00:00:38
3   Control       step_3  0 days 00:01:33
4   Control       step_4  0 days 00:02:09
5      Test       step_0  0 days 00:02:36
6      Test       step_1  0 days 00:00:52
7      Test       step_2  0 days 00:00:48
8      Test       step_3  0 days 00:01:37
9      Test       step_4  0 days 00:02:07


In [19]:
# Pivot the formatted time for a cleaner comparison
pivot_avg_time = avg_time_per_step.pivot(index='process_step', columns='Variation', values='formatted_time')

# Reorder the steps (optional)
step_order = ['step_0', 'step_1', 'step_2', 'step_3', 'step_4']
pivot_avg_time = pivot_avg_time.reindex(step_order)

# Display it
print(pivot_avg_time)


Variation             Control             Test
process_step                                  
step_0        0 days 00:03:19  0 days 00:02:36
step_1        0 days 00:00:56  0 days 00:00:52
step_2        0 days 00:00:38  0 days 00:00:48
step_3        0 days 00:01:33  0 days 00:01:37
step_4        0 days 00:02:09  0 days 00:02:07


##### DROP REPEATED STEPS FOR SAME VISIT ID:

In [21]:
# Step 1: Sort and keep only first appearance of each step per visit
df_first_occurrence = df_ab_time.sort_values(by=['visit_id', 'process_step', 'date_time']) \
    .drop_duplicates(subset=['visit_id', 'process_step'], keep='first')

# Step 2: Convert to seconds
df_first_occurrence['diff_time_seconds'] = df_first_occurrence['diff_time'].dt.total_seconds()



In [22]:
df_first_occurrence['diff_time_seconds'] = df_first_occurrence['diff_time'].dt.total_seconds()


In [23]:
# Step 1: Sort and keep only first appearance of each step per visit
df_first_occurrence = df_time.sort_values(by=['visit_id', 'process_step', 'date_time']) \
    .drop_duplicates(subset=['visit_id', 'process_step'], keep='first')

# Step 2: Convert timedelta to seconds
df_first_occurrence['diff_time_seconds'] = df_first_occurrence['diff_time'].dt.total_seconds()

# Step 3: Group and calculate average time per step
avg_time_first_only = df_first_occurrence.groupby(['process_step', 'Variation'])['diff_time_seconds'].mean().reset_index()

# Step 4: Pivot for comparison
pivot_first_only = avg_time_first_only.pivot(index='process_step', columns='Variation', values='diff_time_seconds').reset_index()

# Step 5: Export if needed
pivot_first_only.to_csv("avg_time_per_step_first_only.csv", index=False)

# Show it
pivot_first_only



Variation,process_step,Control,Test
0,step_0,238.406567,153.982695
1,step_1,57.801654,47.429071
2,step_2,35.930871,38.902694
3,step_3,99.441310,92.975367
4,step_4,128.803766,106.547434


In [30]:
# Step 5: Melt to long format and export
avg_time_melted = avg_time_first_only  # Already in long format

avg_time_melted.to_csv(
    r'C:\Users\manoe.MC_ASUS\Documents\IRON HACK BOOTCAMP\DAFT_Feb 2025\projects\Project 5\Project-5---Vanguard\1_data\tableau\avg_time_per_step.csv',
    index=False
)


### HYPOTHESIS TEST AVG TIME PER STEP KPI:

In [26]:
for step in df_first_occurrence['process_step'].unique():
    step_data = df_first_occurrence[df_first_occurrence['process_step'] == step]

    control = step_data[step_data['Variation'] == 'Control']['diff_time_seconds']
    test = step_data[step_data['Variation'] == 'Test']['diff_time_seconds']

    variance_results.append({
        'step': step,
        'control_variance': control.var(),
        'test_variance': test.var(),
        'control_std': control.std(),
        'test_std': test.std()
    })

df_variance = pd.DataFrame(variance_results).sort_values('step')
df_variance


,step,control_variance,test_variance,control_std,test_std
0,step_0,201792.997403,96188.129064,449.213755,310.142111
1,step_1,21726.802624,13576.579812,147.400145,116.518581
2,step_2,3041.074259,3391.201605,55.145936,58.234024
3,step_3,8333.317256,7763.830349,91.287005,88.112600
4,step_4,26231.803964,27007.777432,161.962354,164.340432


In [27]:
from scipy.stats import ttest_ind

results = []

for step in df_first_occurrence['process_step'].unique():
    step_data = df_first_occurrence[df_first_occurrence['process_step'] == step]
    
    control = step_data[step_data['Variation'] == 'Control']['diff_time_seconds']
    test = step_data[step_data['Variation'] == 'Test']['diff_time_seconds']
    
    # Welch’s t-test (unequal variance)
    t_stat, p_val = ttest_ind(control, test, equal_var=False)
    
    results.append({
        'step': step,
        't_statistic': t_stat,
        'p_value': p_val,
        'mean_control': control.mean(),
        'mean_test': test.mean()
    })

df_results = pd.DataFrame(results).sort_values('step')
df_results


,step,t_statistic,p_value,mean_control,mean_test
0,step_0,12.363419,8.389600e-35,238.406567,153.982695
1,step_1,4.590385,4.475808e-06,57.801654,47.429071
2,step_2,-5.478875,4.304317e-08,35.930871,38.902694
3,step_3,7.145478,9.129944e-13,99.441310,92.975367
4,step_4,12.427284,2.237422e-35,128.803766,106.547434


##### CONCLUSION: All p-values are below 0.00001, meaning we reject the null hypothesis in every case.



### KPI 3 Error rate per group

In [32]:
# Step 1: Filter to Test and Control clients
df_ab = df_web_cleaned.merge(
    df_final_experiment_clients[['client_id', 'Variation']],
    on='client_id',
    how='inner'
)

# Step 2: Count how many times each step appears per visit
step_counts = df_ab.groupby(['visit_id', 'process_step']).size().reset_index(name='count')

# Step 3: Identify visits where any step appears more than once
repeated_steps = step_counts[step_counts['count'] > 1]['visit_id'].unique()

# Step 4: Flag those visits as errors
df_ab_visits = df_ab[['visit_id', 'Variation']].drop_duplicates()
df_ab_visits['error'] = df_ab_visits['visit_id'].isin(repeated_steps)

# Step 5: Compute error rate per Variation
error_rate = df_ab_visits.groupby('Variation')['error'].mean().reset_index()
error_rate['formatted'] = error_rate['error'].apply(lambda x: f"{x:.2%}")

# Show results
print("Error Rate per Variation:")
print(error_rate[['Variation', 'formatted']])


Error Rate per Variation:
  Variation formatted
0   Control    37.78%
1      Test    45.26%


In [33]:
error_rate.to_csv(
    r'C:\Users\manoe.MC_ASUS\Documents\IRON HACK BOOTCAMP\DAFT_Feb 2025\projects\Project 5\Project-5---Vanguard\1_data\tableau\error_rate_overall.csv',
    index=False
)

